In [1]:
import pandas as pd
import numpy as np 
import re
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
train = pd.read_csv("../dataset/train.csv")
test = pd.read_csv("../dataset/test.csv")



In [3]:
submission = test['UID']

In [17]:
y=train['score']

In [19]:
train.head()

,comment,parent_comment,year,month,word_count_c,word_count_p,avg_word_c,avg_word_p,stopwords_c,stopwords_p
0,nc nh,yeah get argument point id prefer lived nc well,2016,10,3,17,2.666667,3.764706,1,8
1,know west teams play west teams east teams right?,blazers mavericks wests seed even carry good e...,2016,11,14,27,4.357143,4.000000,5,11
2,underdogs earlier today since gronks announcem...,theyre favored win,2016,09,19,4,5.421053,5.000000,7,1
3,meme isnt funny none new york nigga ones,deadass dont kill buzz,2016,10,12,5,4.083333,4.400000,4,1
4,could use one tools,yep confirm saw tool use made boy easports mut,2016,12,7,18,3.571429,3.777778,3,10


In [5]:
test.head()

,UID,comment,date,parent_comment
0,Te-1,"Brb, gonna farm my homu",2016-10,So now I KNOW I've sold my soul to the Devil...
1,Te-2,TIL Russia is in Europe,2016-10,The entire CONTINENT of Europe? But arent they...
2,Te-3,Glad they filmed this with an 8mm.,2016-10,Girl drinks whole beer without using her hands
3,Te-4,But what about that cinematic feel?,2016-10,MRW someone defends 24 FPS over 60
4,Te-5,32 spots left as of today.,2016-10,Are there still spots open for this?


In [6]:
train['year']=train['date'].str[:4].astype(object)
train['month']=train['date'].str[5:].astype(object)

In [7]:
test['year']=test['date'].str[:4].astype(object)
test['month']=test['date'].str[5:].astype(object)

In [8]:
train.drop(['UID','date','score'],axis=1,inplace=True)
test.drop(['UID','date'],axis=1,inplace=True)


In [9]:
def _removeNonAscii(s): 
    return "".join(i for i in s if ord(i)<128)

def clean_text(text):
    
    text = text.lower()
    text = re.sub(r"\\", "", text)
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)
    text = re.sub('[^a-zA-Z ?!]+', " ", text)
    text = re.sub(r'[0-9]+', '', text)
    text = _removeNonAscii(text)
    text = text.strip()
    return text

In [11]:
train['word_count_c'] = train['comment'].apply(lambda x: len(str(x).split(" ")))
test['word_count_c'] = test['comment'].apply(lambda x: len(str(x).split(" ")))

train['word_count_p'] = train['parent_comment'].apply(lambda x: len(str(x).split(" ")))
test['word_count_p'] = test['parent_comment'].apply(lambda x: len(str(x).split(" ")))



def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

train['avg_word_c'] = train['comment'].apply(lambda x: avg_word(x))
test['avg_word_c'] = train['comment'].apply(lambda x: avg_word(x))

train['avg_word_p'] = train['parent_comment'].apply(lambda x: avg_word(x))
test['avg_word_p'] = train['parent_comment'].apply(lambda x: avg_word(x))



In [12]:
# Cleaning the data
train['comment']=train.apply(lambda row: clean_text(row['comment']), axis=1)
test['comment']=test.apply(lambda row: clean_text(row['comment']), axis=1)


train['parent_comment']=train.apply(lambda row: clean_text(row['parent_comment']), axis=1)
test['parent_comment']=test.apply(lambda row: clean_text(row['parent_comment']), axis=1)


In [13]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

train['stopwords_c'] = train['comment'].apply(lambda x: len([x for x in x.split() if x in stop_words]))
train['stopwords_p'] = train['parent_comment'].apply(lambda x: len([x for x in x.split() if x in stop_words]))


test['stopwords_c'] = test['comment'].apply(lambda x: len([x for x in x.split() if x in stop_words]))
test['stopwords_p'] = test['parent_comment'].apply(lambda x: len([x for x in x.split() if x in stop_words]))


In [14]:
train['comment'] = train['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))
train['parent_comment'] = train['parent_comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))


test['comment'] = test['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))
test['parent_comment'] = test['parent_comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))




In [15]:
train.head()

,comment,parent_comment,score,year,month,word_count_c,word_count_p,avg_word_c,avg_word_p,stopwords_c,stopwords_p
0,nc nh,yeah get argument point id prefer lived nc well,2,2016,10,3,17,2.666667,3.764706,1,8
1,know west teams play west teams east teams right?,blazers mavericks wests seed even carry good e...,-4,2016,11,14,27,4.357143,4.000000,5,11
2,underdogs earlier today since gronks announcem...,theyre favored win,3,2016,09,19,4,5.421053,5.000000,7,1
3,meme isnt funny none new york nigga ones,deadass dont kill buzz,-8,2016,10,12,5,4.083333,4.400000,4,1
4,could use one tools,yep confirm saw tool use made boy easports mut,6,2016,12,7,18,3.571429,3.777778,3,10


In [16]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.20, random_state=314, stratify=y)

comment            object
parent_comment     object
score               int64
year               object
month              object
word_count_c        int64
word_count_p        int64
avg_word_c        float64
avg_word_p        float64
stopwords_c         int64
stopwords_p         int64
dtype: object

In [1]:
from catboost import CatBoostRegressor

In [ ]:
model = CatBoostRegressor(iterations=500,
                             depth=8,
                             bagging_temperature = 0.2,
                             use_best_model=True,
                             random_seed=1993,
                             early_stopping_rounds=50)


In [ ]:
cate_features_index = np.where((train.dtypes != float) & (train.dtypes != np.dtype('int64')))[0]
model_cat=model.fit(train,y,cat_features=cate_features_index,eval_set=(train, y),plot=True)



In [ ]:
submission['Score'] = pd.DataFrame(model_cat.predict(test))
submission.to_csv('.../cat_boost.csv',sep=',' , index=None)